In [10]:
import ipathapy as ipath
import pandas as pd 
import numpy as np 

## Example 1 - Simple metabolite retrieval 

Say you are interested in looking at a certain set of molecular formulas, which metabolites they correspond to and where they are located in a metabolic network. Provide the list of molecular formulas and pass them to `ipath.kegg.batch_retrieval`. This function produces a `pd.DataFrame` object that contains the molecular formula and corresponding KEGG ID. 

To get the names of the compounds, retrieve the ID-Name pairs of all compounds in KEGG with `ipath.kegg.all_compounds` and merge the two dataframes with `pd.merge` 

In [2]:
# Simple example, starting from here
molformulas = [
    'C5H5N5', # Adenine and others 
    'C6H12O6', # Hexoses
    'C4H6O4', # Succinate and others
]

molformula_keggID = ipath.kegg.batch_retrieval(molformulas)
keggID_name = ipath.kegg.all_compounds()

# Left join to get all names that are associated with the molecular formulas of interest. 
df_annotated = pd.merge(molformula_keggID, keggID_name, on = 'id', how = 'left')

# See what the output looks like 
df_annotated.sample(10)

100%|██████████| 3/3 [00:03<00:00,  1.26s/it]


,id,molecular_formula,name
8,C00267,C6H12O6,alpha-D-Glucose
34,C06466,C6H12O6,D-Idose
1,C00031,C6H12O6,D-Glucose
14,C00962,C6H12O6,beta-D-Galactose
21,C01720,C6H12O6,L-Fuconate
18,C01582,C6H12O6,Galactose
44,C21066,C6H12O6,D-Galactofuranose
36,C06468,C6H12O6,D-Psicose
30,C06152,C6H12O6,muco-Inositol
50,C21761,C4H6O4,(3R)-3-Hydroxy-4-oxobutanoate


Now, highlight all of these compounds in the metabolic pathway map of ipath3, using the module `ipathapy.ipath`

If you do not specify any color or size, the default arguments are `color = '#FF0000'` (red) and `size = 10` (px). However, you can either customize this by providing your own values OR can provide iterables with the same length as the id-iterable so that you can customize the layout for *every* compound individually (see advanced example). 

In [3]:
keggIDs = df_annotated['id']
print(ipath.ipath.make_ipath_selection(keggIDs, colors = '#2471A3', sizes = 20))

# To save as textfile: 
# ipath.ipath.make_ipath_selection(keggIDs, colors = '#2471A3', sizes = 20, save = 'ipath-input-example.txt')

C00147 #2471A3 W20
C00031 #2471A3 W20
C00095 #2471A3 W20
C00124 #2471A3 W20
C00137 #2471A3 W20
C00159 #2471A3 W20
C00221 #2471A3 W20
C00247 #2471A3 W20
C00267 #2471A3 W20
C00737 #2471A3 W20
C00738 #2471A3 W20
C00764 #2471A3 W20
C00795 #2471A3 W20
C00936 #2471A3 W20
C00962 #2471A3 W20
C00984 #2471A3 W20
C01452 #2471A3 W20
C01487 #2471A3 W20
C01582 #2471A3 W20
C01680 #2471A3 W20
C01719 #2471A3 W20
C01720 #2471A3 W20
C01825 #2471A3 W20
C01906 #2471A3 W20
C01934 #2471A3 W20
C02209 #2471A3 W20
C02336 #2471A3 W20
C02782 #2471A3 W20
C05003 #2471A3 W20
C06151 #2471A3 W20
C06152 #2471A3 W20
C06153 #2471A3 W20
C06464 #2471A3 W20
C06465 #2471A3 W20
C06466 #2471A3 W20
C06467 #2471A3 W20
C06468 #2471A3 W20
C08351 #2471A3 W20
C08356 #2471A3 W20
C10906 #2471A3 W20
C15923 #2471A3 W20
C19891 #2471A3 W20
C21032 #2471A3 W20
C21050 #2471A3 W20
C21066 #2471A3 W20
C21523 #2471A3 W20
C22502 #2471A3 W20
C00042 #2471A3 W20
C02170 #2471A3 W20
C10900 #2471A3 W20
C21761 #2471A3 W20



Go to [ipath3](https://pathways.embl.de/tools.cgi) and paste this selection into the field `selection` (the automatic web API was unfortunately disabled). Specify further parameters (it is recommended to change the value of `default_color` to a less intense gray, e.g. `#cccccc`). In the example, the species filter `hsa` was used to only display the human metabolism. The output should look like this: 


<img src="example1.png" alt="Output for example 1" width="600">

As you can see, most metabolites were not found in the map. If you want to know which entities were not found, you can use the `Selection Match Statistics` tab in ipath3. For example, you could download its output as .csv, import the data into a jupyter notebook and join the information with the input data using  `pd.merge` to see which metabolites were found. This is also an easy + quick way to see which isomers in your dataset are most plausible. 

## Example 2 - Advanced 

If you perform a metabolic experiment in which you compare different conditions, you probably get some kind of metric for the change/difference between different conditions. `ipathapy` can be used to visualize these quantitative measures on a metabolic pathway map.  

In [4]:
# Create a dataframe with some example metabolites from MS1 experiments (ions with adducts) and some quantitative measures. 
df = pd.DataFrame(
    {
    'ions': {0: 'C5H5N5-H', 1: 'C6H12O6-H', 2: 'C4H6O4-H'}, # Adenine, Hexoses, Succinate
    'scores': {0: 3.7321227, 1: 26.354689, 2: -1.7803773},
    'logfoldchanges': {0: 0.3070414, 1: 0.51227486, 2: -6.6116753},
    'pvals': {0: 0.0001898730023755, 1: 4.535010108462715e-153, 2: 0.075014248841717},
    'pvals_adj': {0: 0.0003114967227913, 1: 3.937716094177382e-152, 2: 0.1019277579681346}
    }
    )
df

,ions,scores,logfoldchanges,pvals,pvals_adj
0,C5H5N5-H,3.732123,0.307041,1.898730e-04,3.114967e-04
1,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152
2,C4H6O4-H,-1.780377,-6.611675,7.501425e-02,1.019278e-01


### Preprocessing

The generated dataframe corresponds to a standardized output of a `scanpy` enrichment analysis, `ipathapy` has a preprocessing module that allows you to quickly filter this standardized data for significant/interesting metabolites. `ipathapy.preprocess.parse_ranked_ions` automatically extracts the molecular formula from the provided ion-formula and allows filtering for 1) significance of the enrichment/change 2) magnitude of change/value.   

Let's filter for metabolites that exhibit a significant (`p<0.001`) change in their signals between the 2 compared conditions. In this case, the molecule `C5H6O4` is removed from the dataset, because its intensity levels do not significantly change between conditions. The function further extracts molecular formula and adduct with the regular expression `"(.*)[\+|\-](.*)"` from the `ion` column. All column names can be modified so that this function can also be used in cases where the column names are slightly different. 

In [5]:
# filter for only significantly enriched metabolites + add molecular formula
# The ion C4H6O4-H is removed, because it does not meet the p-value cutoff criteria 
df = ipath.preprocess.parse_ranked_ions(df, 
                                        pcutoff = 1e-3,
                                        changecutoff = 0,  # Only show enriched metabolites 
                                        ion_column = 'ions',
                                        logchange_column='logfoldchanges', 
                                        pval_column = 'pvals_adj'
                                        )
df

,ions,scores,logfoldchanges,pvals,pvals_adj,molecular_formula,adduct
0,C5H5N5-H,3.732123,0.307041,1.898730e-04,3.114967e-04,C5H5N5,H
1,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H


## KEGG Annotation

We now associate the quantitative measures with their KEGG IDs. Again, there is a streamlined function (`ipathapy.kegg.annotate_data`) for that, that automatically annotates the dataframe accordingly.

Commonly, KEGG will return multiple isomers for one molecular formula

In [6]:
df_annotated = ipath.kegg.annotate_data(df, molecular_formula_column='molecular_formula')
df_annotated.sample(10)

100%|██████████| 2/2 [00:01<00:00,  1.25it/s]


,ions,scores,logfoldchanges,pvals,pvals_adj,molecular_formula,adduct,id,name
32,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C06464,D-Altrose
14,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00962,beta-D-Galactose
34,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C06466,D-Idose
44,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C21066,D-Galactofuranose
9,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00737,D-Aldose
11,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00764,D-Sorbose
26,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C02336,beta-D-Fructose
33,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C06465,D-Gulose
20,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C01719,L-Fructose
35,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C06467,D-Talose


However, if the data looks slightly different, you can always use the `ipathapy.kegg.batch_retrieval` function from example 1 in combination with some join functionalities of pandas. For the given dataframe, you would get the same result as `ipathapy.kegg.annotate_data` with: 

```python 

molformulas = df['molecular_formula'].unique() #(.unique() is strictly speaking not necessary)

molformula_keggID = ipath.kegg.batch_retrieval(molformulas)
keggID_name = ipath.kegg.all_compounds()

# Generate a dataframe that associates KEGG IDs with their molecular formulas and the respective name
df_kegg = pd.merge(molformula_keggID, keggID_name, on = 'id', how = 'left')

# Merge quantitative measures of df with df_kegg and get the annotated dataframe
df_annotated = pd.merge(df, df_kegg, how = 'outer', on='molecular_formula')

```

### Confidence

As mentioned above, you cannot be sure which isomere was actually detected in your sample if you only know its molecular weight (or molecular formula respectively). To have a first guess for the confidence `ipathapy` can calculate the *redundancy* of the molecular formula, meaning how many distinct KEGG IDs correspond to one molecular formula. For this, you can use the function `ipath.kegg.calculate_redundancy`. The higher the redundancy, the more molecules were retrieved from KEGG and the less confident you can be about the assignment to a certain species. 

In [7]:
df_annotated = ipath.kegg.calculate_redundancy(df_annotated, molecular_formula_column='molecular_formula')
df_annotated.head(10)

,ions,scores,logfoldchanges,pvals,pvals_adj,molecular_formula,adduct,id,name,redundancy
0,C5H5N5-H,3.732123,0.307041,1.898730e-04,3.114967e-04,C5H5N5,H,C00147,Adenine,1
1,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00031,D-Glucose,46
2,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00095,D-Fructose,46
3,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00124,D-Galactose,46
4,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00137,myo-Inositol,46
5,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00159,D-Mannose,46
6,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00221,beta-D-Glucose,46
7,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00247,L-Sorbose,46
8,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00267,alpha-D-Glucose,46
9,C6H12O6-H,26.354689,0.512275,4.535010e-153,3.937716e-152,C6H12O6,H,C00737,D-Aldose,46


In this specific example, there is only one molecule with the molecular formula $\mathrm{C}_5\mathrm{H}_5\mathrm{N}_5$ (Adenine), but 46 different molecules corresponding to hexoses $\mathrm{C}_6\mathrm{H}_{12}\mathrm{O}_6$ were found. 

### Layout

The layout module helps you to encode your quantitative data in colors or sizes of your entities in the ipath metabolic map. In this example, we will color our data according to our confidence that the respective molecule is the correct one (column `redundancy`) and change the size of the entity according to the `logfoldchange`.  

At the moment, you have to specify the colors as tuples in the RGB format. A nice website to select suitable colors can be found [here](https://htmlcolorcodes.com/)

**You can also normalize colors according to different schemes. At the moment, Z-score, log-normalization and quantil assignment are supported normalization measures**


In [35]:
# Since there are only two molecular formulas in the dataset, we overwrite the actual data with random values to get more diversity 
np.random.seed(42)
df_annotated['redundancy'] = np.random.randint(0, 100, size = df_annotated['redundancy'].shape) # generate random numbers in the range [0..100) with the dimensions of the original data

# Add colors to dataframe. This is not necessary, the colors can also be stored as list in a variable 
df_annotated['colors'] = ipath.layout.color(df = df_annotated, column='redundancy', colors = [(231, 76, 60), (244, 246, 246), (46, 134, 193)])

The sizes of the nodes are assigned according to their logfold-change between conditions. The range is set between minimally 3 px and maximally 20 px. Furthermore, the data is binned into 10 bins (the function extracts the number of bins from quantilXX via regex). 

In [40]:
# Since there are only two molecular formulas in the dataset, we overwrite the actual data with random values to get more variability
df_annotated['logfoldchanges'] = 2*np.random.random(size = df_annotated['redundancy'].shape) # generate random numbers in the range [0..2) with the dimensions of the original data
df_annotated['sizes'] = ipath.layout.size(df_annotated, 'logfoldchanges', minsize=10, maxsize=30, normalization='quantil10')

The dataframe now looks like this: 

In [41]:
df_annotated.head(5)

,ions,scores,logfoldchanges,pvals,pvals_adj,molecular_formula,adduct,id,name,redundancy,colors,sizes
0,C5H5N5-H,3.732123,0.854216,1.898730e-04,3.114967e-04,C5H5N5,H,C00147,Adenine,51,#eff3f4,21
1,C6H12O6-H,26.354689,1.636030,4.535010e-153,3.937716e-152,C6H12O6,H,C00031,D-Glucose,92,#4a96c8,27
2,C6H12O6-H,26.354689,1.721461,4.535010e-153,3.937716e-152,C6H12O6,H,C00095,D-Fructose,14,#ea796d,27
3,C6H12O6-H,26.354689,0.013904,4.535010e-153,3.937716e-152,C6H12O6,H,C00124,D-Galactose,71,#9fc6df,10
4,C6H12O6-H,26.354689,1.021495,4.535010e-153,3.937716e-152,C6H12O6,H,C00137,myo-Inositol,60,#cbdfeb,21


## ipath

Finally, we generate again the input for ipath3. Additonally, we would like to highlight important metabolic pathways. This can be achieved with the `highlight` keyword in the `ipath.ipath.make_ipath_selection` function which as been introduced in example 1. 

`highlight` accepts a json like format which specifies the entity with its KEGG ID (`keggID`), the desired color (`color`) and size (`size`). Both the `color` and `size` argument are optional

In [42]:
highlights = [
    {'keggID': '00010','color': '#13878D', 'size':5}, # Glycolysis
    {'keggID': '00030','color': '#76D7C4', 'size':5}, # Pentose Phosphate Pathway
    {'keggID': '00020','color': '#2471A3', 'size':5}, # TCA cycle 
    {'keggID': '00061','color': '#FAD7A0', 'size':5}, # Fatty acid degradation
    {'keggID': '00071','color': '#DC7633', 'size':5}  # Fatty acid synthesis
    ]

selection = ipath.ipath.make_ipath_selection(ids = df_annotated['id'], 
                                colors = df_annotated['colors'], 
                                sizes = df_annotated['sizes'], 
                                highlight = highlights)

print(selection)

00010 #13878D W5
00030 #76D7C4 W5
00020 #2471A3 W5
00061 #FAD7A0 W5
00071 #DC7633 W5
C00147 #eff3f4 W21
C00031 #4a96c8 W27
C00095 #ea796d W27
C00124 #9fc6df W10
C00137 #cbdfeb W21
C00159 #ec8d84 W18
C00221 #72acd3 W12
C00247 #62a3cf W12
C00267 #93bfdc W18
C00737 #93bfdc W30
C00738 #5ea1cd W16
C00764 #2e86c1 W23
C00795 #ec988f W25
C00936 #e74f3f W18
C00962 #ec9187 W30
C00984 #ebf1f3 W30
C01452 #e74c3c W14
C01487 #5ea1cd W21
C01582 #eeada6 W16
C01680 #f0c8c4 W16
C01719 #e74c3c W10
C01720 #bfd8e7 W23
C01825 #cfe1ec W21
C01906 #ec8d84 W10
C01934 #efb7b1 W16
C02209 #8ebcda W30
C02336 #d7e6ee W14
C02782 #ec9187 W12
C05003 #5a9fcc W21
C06151 #f3efee W30
C06152 #529aca W14
C06153 #d3e3ed W25
C06464 #f1d6d3 W27
C06465 #4e98c9 W14
C06466 #cfe1ec W27
C06467 #7eb3d6 W18
C06468 #ea796d W23
C08351 #c7dcea W25
C08356 #c7dcea W23
C10906 #f2e8e6 W12
C15923 #c7dcea W27
C19891 #f4f6f6 W16
C21032 #e3ecf1 W12
C21050 #bfd8e7 W10
C21066 #e74f3f W23
C21523 #f4f6f6 W25
C22502 #e85d4e W10



The final output should look something like this: 

<img src="example2.png" alt="Output for example 2" width="600">
